In [37]:
import pandas as pd
import json
import requests as re
from azure.storage.filedatalake import DataLakeServiceClient


In [45]:
# Get all bike sharing networks
response = re.get("https://api.citybik.es/v2/networks/")
if response.status_code == 200:
    data = response.json()
    # Convert the dict/list to a JSON string
    json_data = json.dumps(data)

In [41]:
storage_account_name = "bikeflowproject"
storage_account_key = "zqgOdUlvtvTRnvgMCV6NQPFh7Qw08D+yufsDepsYzn246/wRdh+wrFO2BS5kNbKYf6Y3umfk/n7s+AStNSE1Rg=="
container_name = "bronzelayer"

In [42]:
service_client = DataLakeServiceClient(
    account_url=f"https://{storage_account_name}.dfs.core.windows.net", 
    credential=storage_account_key
)

In [46]:
try:
    file_system_client = service_client.get_file_system_client(file_system=container_name)
    # directory_client = file_system_client.get_directory_client("bike_data") # Optional folder
    file_client = file_system_client.create_file("city_bikes.json")
    
    # Upload data
    file_client.append_data(data=json_data, offset=0, length=len(json_data))
    file_client.flush_data(len(json_data))
    
    print("Upload successful!")
except Exception as e:
    print(f"Error: {e}")

Upload successful!


In [26]:
print(type(data))

<class 'dict'>


In [17]:
print(json.dumps(data, indent=2))

{
  "networks": [
    {
      "id": "abu-dhabi-careem-bike",
      "name": "Abu Dhabi Careem BIKE",
      "location": {
        "latitude": 24.4866,
        "longitude": 54.3728,
        "city": "Abu Dhabi",
        "country": "AE"
      },
      "href": "/v2/networks/abu-dhabi-careem-bike",
      "company": [
        "Careem"
      ],
      "gbfs_href": "https://dubai.publicbikesystem.net/customer/gbfs/v2/en/gbfs.json"
    },
    {
      "id": "acces-velo-saguenay",
      "name": "Acc\u00e8s V\u00e9lo",
      "location": {
        "latitude": 48.433333,
        "longitude": -71.083333,
        "city": "Saguenay",
        "country": "CA"
      },
      "href": "/v2/networks/acces-velo-saguenay",
      "company": [
        "PBSC Urban Solutions"
      ],
      "gbfs_href": "https://saguenay.publicbikesystem.net/customer/gbfs/v2/gbfs.json"
    },
    {
      "id": "aksu",
      "name": "Aksu",
      "location": {
        "latitude": 41.1664,
        "longitude": 80.2617,
        "city": 

In [31]:
data = data["networks"]
print(type(data))


<class 'list'>


In [32]:
for things in data[:10]:
    print(things)

{'id': 'abu-dhabi-careem-bike', 'name': 'Abu Dhabi Careem BIKE', 'location': {'latitude': 24.4866, 'longitude': 54.3728, 'city': 'Abu Dhabi', 'country': 'AE'}, 'href': '/v2/networks/abu-dhabi-careem-bike', 'company': ['Careem'], 'gbfs_href': 'https://dubai.publicbikesystem.net/customer/gbfs/v2/en/gbfs.json'}
{'id': 'acces-velo-saguenay', 'name': 'Accès Vélo', 'location': {'latitude': 48.433333, 'longitude': -71.083333, 'city': 'Saguenay', 'country': 'CA'}, 'href': '/v2/networks/acces-velo-saguenay', 'company': ['PBSC Urban Solutions'], 'gbfs_href': 'https://saguenay.publicbikesystem.net/customer/gbfs/v2/gbfs.json'}
{'id': 'aksu', 'name': 'Aksu', 'location': {'latitude': 41.1664, 'longitude': 80.2617, 'city': '阿克苏市 (Aksu City)', 'country': 'CN'}, 'href': '/v2/networks/aksu', 'company': ['阿克苏公共服务']}
{'id': 'alba', 'name': 'Alba', 'location': {'latitude': 44.716667, 'longitude': 8.083333, 'city': 'Alba', 'country': 'IT'}, 'href': '/v2/networks/alba', 'company': ['Comunicare S.r.l.'], 'sys

In [33]:
print(data.keys())

AttributeError: 'list' object has no attribute 'keys'

In [36]:
df = pd.json_normalize(data)
    
print(f"Total networks: {len(df)}")
print(f"\nAll columns extracted automatically:")
print(df.columns.tolist())
    
print(f"\nFirst 5 rows:")
print(df.head())

Total networks: 788

All columns extracted automatically:
['id', 'name', 'href', 'company', 'gbfs_href', 'location.latitude', 'location.longitude', 'location.city', 'location.country', 'system', 'source', 'ebikes', 'license.name', 'license.url', 'scooters', 'instances']

First 5 rows:
                      id                   name  \
0  abu-dhabi-careem-bike  Abu Dhabi Careem BIKE   
1    acces-velo-saguenay             Accès Vélo   
2                   aksu                   Aksu   
3                   alba                   Alba   
4               albabici               AlbaBici   

                                 href  \
0  /v2/networks/abu-dhabi-careem-bike   
1    /v2/networks/acces-velo-saguenay   
2                   /v2/networks/aksu   
3                   /v2/networks/alba   
4               /v2/networks/albabici   

                                             company  \
0                                           [Careem]   
1                             [PBSC Urban Soluti